In [ ]:
#| default_exp tcp_hello_observer

# TcpHelloObserver

> Class for reading hello_world values over tcp

In [33]:
#| export
import asyncio # tcp server

In [48]:
#| export
class TcpHelloObserver:

    def __init__(self):
        self.x = []
        self.y = []
        self.server = None
        self.__connected = False
    
    def parse_num(num_str : str):
        """Parses a number from a string in the format used by hello_world. The base of the exponentation is always assumed to be 2 and not parsed.
        The exponent must be an integer.
        """
        base = num_str[:num_str.find('*')]
        exponent = num_str[num_str.find('^')+1:]
        return float(base) * (2 ** int(exponent))

    def parse_data(message : str):
        "Returns float (x,y) pair from `message` which follows formatting used by hello_world"
        parts = message.split(' ')
        return (TcpHelloObserver.parse_num(parts[1][:-1]), TcpHelloObserver.parse_num(parts[3]))

    async def __handle_connection(self, reader, writer):
        "Handles an incoming tcp connection by reading lines, parsing them and storing the x and y values"
        self.__connected = True
        while True:
            data = await reader.readline()
            if not data:
                break

            message = data.decode()
            if len(message) > 2: # if not an empty line
                point = TcpHelloObserver.parse_data(message)
                self.x.append(point[0])
                self.y.append(point[1])
            
            await asyncio.sleep(0) # pass control to main
        self.__connected = False

    async def start_server(self, hostname : str, portnum : int):
        "Sets up a server"
        self.server = await asyncio.start_server(
                self.__handle_connection, hostname, portnum)

    async def serve_connection(self):
        "Tries to start a connection with client if one doesn't exist"
        await self.server.start_serving()

    def is_connected(self):
        "Returns true if a client is connected"
        return self.__connected


In [35]:
#| hide
from nbdev.showdoc import *
from fastcore.test import *

# Attributes

|attribute|description|
|----|-----------|
|x| list of floating point x values
|y| list of floating point y values

# Methods

In [36]:
show_doc(TcpHelloObserver.parse_num)

---

### TcpHelloObserver.parse_num

>      TcpHelloObserver.parse_num (num_str:str)

parses a number from a string in the format used by hello_world. The base of the exponentation is always assumed to be 2 and not parsed.
The exponent must be an integer.

In [37]:
test_eq(TcpHelloObserver.parse_num("1.0*2^8"),256)

In [38]:
"Note that the '3' is assumed to be a 2"
TcpHelloObserver.parse_num("1.0*3^8")

256.0

In [39]:
show_doc(TcpHelloObserver.parse_data)

---

### TcpHelloObserver.parse_data

>      TcpHelloObserver.parse_data (message:str)

returns float (x,y) pair from `message` which follows formatting used by hello_world

In [40]:
test_eq(TcpHelloObserver.parse_data("x_value: 1.5*2^10, y_value: 1*2^-1"), (1.5*2**10, 1*2**(-1)))
test_eq(TcpHelloObserver.parse_data("x_value: 1.2566366*2^0, y_value: 1.9316164*2^-1"), (1.2566366*2**0, 1.9316164*2**(-1)))

In [43]:
show_doc(TcpHelloObserver.start_server)

---

### TcpHelloObserver.start_server

>      TcpHelloObserver.start_server (hostname:str, portnum:int)

In [46]:
show_doc(TcpHelloObserver.serve_connection)

---

### TcpHelloObserver.serve_connection

>      TcpHelloObserver.serve_connection ()

In [44]:
show_doc(TcpHelloObserver.is_connected)

---

### TcpHelloObserver.is_connected

>      TcpHelloObserver.is_connected ()

Returns true if a client is connected

In [41]:
#| hide
import nbdev; nbdev.nbdev_export()